In [21]:
%load_ext autoreload
%autoreload 2
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler,OneHotEncoder
from xgboost import XGBClassifier, XGBRegressor
import pathlib
import glob
from sklearn.metrics import accuracy_score,precision_score,median_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin, BaseEstimator
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pickle import dump,load

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
dir_path = pathlib.Path('./train/')
csv_path = dir_path.joinpath('*.csv')

print(csv_path)
all_files = glob.glob(csv_path.as_posix())
df=None
for f in all_files:
    if  df is None:
        df = pd.read_csv(f)
    else:
        df = pd.concat([df,pd.read_csv(f)])
# df = df.drop(['market cap','volume','operating cash flow'],axis=1)

train/*.csv


In [26]:
df.shape

(14032, 29)

In [27]:
df.describe()

,market_cap,price_per_sale,current_price,price_per_book,return_on_assets,return_on_equity,revenue_per_share,quarterly_revenue_growth,total_cash_per_share,debt_equity_ratio,...,book_value_per_dollar,earnings_per_dollar,revenue_per_dollar,price_to_free_cash_flow,sp_52_week_change,stock_52_week_change,FCF_yield,EV_/_FCF,missing,trend_sp
count,1.403200e+04,14013.000000,14032.000000,13704.000000,13421.000000,13150.000000,14032.000000,13770.000000,14032.000000,9134.000000,...,1.403200e+04,1.403200e+04,1.403200e+04,1.403200e+04,14030.000000,13524.000000,14032.000000,1.403200e+04,14032.000000,14032.000000
mean,5.036113e+06,38.960807,26.203893,4.830293,3.890072,5.534572,20.908944,69.604960,4.271441,122.568211,...,6.347562e+07,6.665787e+06,1.205561e+08,1.295692e+06,9.550793,13.679428,0.158491,1.547752e+06,0.856186,-0.518341
std,2.050327e+07,1489.562448,30.938145,79.827902,20.772009,255.135755,37.583345,2418.458874,17.927237,5084.540992,...,2.421220e+08,4.192727e+07,5.105751e+08,1.250485e+07,5.014822,45.466600,11.199093,1.739387e+07,1.141783,10.806340
min,0.000000e+00,0.030000,1.770500,0.040000,-2035.230000,-10458.540000,-2.570000,-99.400000,0.000000,0.000000,...,-2.516224e+09,-1.089886e+09,-1.769307e+07,-3.273056e+04,-1.070000,-85.320000,-35.247847,-2.198900e+08,0.000000,-76.990000
25%,1.635500e+05,0.960000,10.357500,1.340000,1.170000,3.180000,4.042000,2.100000,0.510000,0.236000,...,5.329778e+06,1.660998e+05,4.836914e+06,7.207113e+00,8.770000,-8.700000,0.000036,7.847566e+00,0.000000,-5.560000
50%,5.955350e+05,2.070000,19.320000,2.050000,4.160000,10.230000,9.614500,11.300000,1.698000,0.749000,...,1.490032e+07,1.026451e+06,1.881976e+07,1.613372e+01,9.140000,6.860000,0.040313,1.820217e+01,0.000000,-1.120000
75%,2.330000e+06,4.300000,34.240000,3.340000,8.030000,17.367500,24.047000,25.700000,4.084000,5.109000,...,3.972553e+07,4.171715e+06,6.635271e+07,4.018761e+01,14.510000,26.770000,0.081845,4.619002e+01,2.000000,4.112500
max,5.948800e+08,109905.040000,751.000000,7730.000000,119.610000,14573.690000,900.699000,186204.500000,599.496000,340533.880000,...,8.418672e+09,8.995291e+08,2.126071e+10,5.948800e+08,17.830000,1000.000000,1323.773585,7.621400e+08,7.000000,169.800000


In [32]:
df.trend_sp.describe()

count    14032.000000
mean        -0.518341
std         10.806340
min        -76.990000
25%         -5.560000
50%         -1.120000
75%          4.112500
max        169.800000
Name: trend_sp, dtype: float64

In [33]:
df.shape

(14032, 29)

In [35]:
# With all features
numerical_cols = df.select_dtypes('number').columns.drop('trend_sp')
id_feature = ['Stock_code']
categorical_cols = df.select_dtypes(exclude = ['number','boolean']).columns.drop(['Stock_code'])
target_feature = ['trend_sp']

In [37]:
X = df.drop(target_feature + id_feature,axis=1)
y = df[target_feature]

In [39]:
print(X.shape, y.shape)

(14032, 27) (14032, 1)


In [40]:
X.columns

Index(['market_cap', 'price_per_sale', 'current_price', 'price_per_book',
       'return_on_assets', 'return_on_equity', 'revenue_per_share',
       'quarterly_revenue_growth', 'total_cash_per_share', 'debt_equity_ratio',
       'book_value_per_share', 'operating_cash_flow', 'volume',
       'Price_per_Earnings_ratio', 'earnings_per_shares', 'sector', 'industry',
       'total_cash_per_dollar', 'book_value_per_dollar', 'earnings_per_dollar',
       'revenue_per_dollar', 'price_to_free_cash_flow', 'sp_52_week_change',
       'stock_52_week_change', 'FCF_yield', 'EV_/_FCF', 'missing'],
      dtype='object')

## Calculate the average score of topK predictions from random split
### You can test with different K and iterations

In [67]:
def average_score(X,y,topK = 10, iterations = 5):
    scores = []

    for i in range(iterations):
        

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2)

        class Debug(BaseEstimator, TransformerMixin):

            def transform(self, X):
                print(X.shape)
                display(pd.DataFrame(X))
        #         self.shape = shape
                # what other output you want
                return X

            def fit(self, X, y=None, **fit_params):
                return self

        # Preprocessing for numerical data
        numerical_transformer =  Pipeline(steps=[
        #     ('imputer', SimpleImputer(strategy='median')),
            ('std', StandardScaler())
        ])
        # Preprocessing for categorical data
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant',fill_value='nan')),
            ('oe', OneHotEncoder(handle_unknown='ignore'))
        ])

        # Bundle preprocessing for numerical and categorical data
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_cols),
                ('cat', categorical_transformer, categorical_cols)
            ])

        # Binary Target
#         model = XGBClassifier(n_estimators=100,max_depth=6)

        model = XGBRegressor()

        # Bundle preprocessing and modeling code in a pipeline
        my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
        #                               ("debug", Debug()),
                                      ('model', model)
                                     ])

        X_train_after = preprocessor.fit_transform(X_train)
        X_test_after = preprocessor.transform(X_test)

        model.fit(X_train_after,y_train)

        y_train_hat = model.predict(X_train_after)
        y_test_hat = model.predict(X_test_after)

    #     print(mean_squared_error(y_train, y_train_hat))
    #     print(mean_squared_error(y_test, y_test_hat))


        y_test_hat_reindex = pd.Series(y_test_hat,index=y_test.index).reset_index()
        y_test_reindex = pd.DataFrame(y_test).reset_index()

#         top_y_test_hat =  y_test_hat_reindex.iloc[y_test_hat_reindex[0].argsort()[-topK:]]
      
        top_y_test = y_test_reindex.iloc[y_test_hat_reindex[0].argsort()[-topK:]]
#         print(top_y_test['trend_sp'].mean())
        scores.append(top_y_test['trend_sp'].mean())
    return (pd.DataFrame(scores).describe())


In [70]:
average_score(X,y,topK=30,iterations=30)

,0
count,30.000000
mean,3.560833
std,2.472193
min,-1.360000
25%,1.991417
50%,3.623000
75%,5.239833
max,8.955667


## Generate Predictions

### Train the final model with all data

In [71]:

def train_model(X,y):

    # Preprocessing for numerical data
    numerical_transformer =  Pipeline(steps=[
    #     ('imputer', SimpleImputer(strategy='median')),
        ('std', StandardScaler())
    ])
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant',fill_value='nan')),
        ('oe', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])
    

    
    model = XGBRegressor()

    # Bundle preprocessing and modeling code in a pipeline
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
    #                               ("debug", Debug()),
                                  ('model', model)
                                 ])

    X_train_after = preprocessor.fit_transform(X)
    
    model.fit(X_train_after,y)
    
    model.save_model('model/xgboost_0311.json')
    dump(preprocessor, open('model/preprocessor.pkl', 'wb'))


In [72]:
train_model(X,y)